In [2]:
import numpy as np
def log_std(a):
    a=a
    if a<=0.0 or a==float("inf"):
        a=np.nan
    return a

df['Volume']=df['Volume'].apply(log_std)
df['VWAP']=df['VWAP'].apply(log_std)
df_std=pd.DataFrame()
for i in range(len(df['Asset_ID'].unique())):
    data=df[df['Asset_ID']==i].set_index('timestamp')
    data=data.reindex(range(data.index[0],data.index[-1]+60,60),method='pad')
    data=data.fillna(method='pad')
    data=data.reset_index()
    after_15=np.array(data['Close'].iloc[15:].values).T
    now=np.array(data['Close'].iloc[:-15].values).T
    target=(after_15/now-now/now).tolist()
    while True:
        target.append(target[-1])
        if len(target)==len(data['timestamp']):
            break
    data['target']=target
    data=data.fillna(method='pad')
    df_std=df_std.append(data,ignore_index=True)
df_std

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,target
0,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,-0.015240
1,1514764920,0,7.0,8.530000,8.530000,8.514500,8.514500,7.139000e+01,8.520215,-0.015875,-0.014399
2,1514764980,0,45.0,8.506500,8.529900,8.484800,8.484800,1.546820e+03,8.501394,-0.015410,-0.015875
3,1514765040,0,14.0,8.500900,8.506600,8.474400,8.500900,1.258000e+02,8.479810,-0.012524,-0.015410
4,1514765100,0,5.0,8.500700,8.500700,8.456000,8.456000,1.250100e+02,8.458435,-0.005940,-0.012524
...,...,...,...,...,...,...,...,...,...,...,...
26080038,1632182160,13,477.0,0.091435,0.091500,0.091120,0.091221,2.613704e+06,0.091340,0.004163,0.004163
26080039,1632182220,13,980.0,0.091131,0.091195,0.090802,0.090863,4.991573e+06,0.090991,0.004163,0.004163
26080040,1632182280,13,720.0,0.090853,0.090940,0.090740,0.090859,3.699102e+06,0.090838,0.004163,0.004163
26080041,1632182340,13,686.0,0.090941,0.091450,0.090860,0.091326,3.241507e+06,0.091193,0.004163,0.004163


In [3]:
df=0
#del df_std['timestamp_std']
df_std['Asset_ID']=df_std['Asset_ID'].astype(np.int8)
df_std['Count']=df_std['Count'].astype(np.int32)
df_std['Open']=df_std['Open'].astype(np.float32)
df_std['High']=df_std['High'].astype(np.float32)
df_std['Low']=df_std['Low'].astype(np.float32)
df_std['Close']=df_std['Close'].astype(np.float32)
df_std['Volume']=df_std['Volume'].astype(np.float32)
df_std['VWAP']=df_std['VWAP'].astype(np.float32)

In [4]:
asset_details = pd.read_csv(r"D:\Desktop\asset_details.csv")

#create dictionnary of weights
dict_weights = {}
for i in range(asset_details.shape[0]):
    dict_weights[asset_details.iloc[i,0]] = asset_details.iloc[i,1]
df_std['weights'] = df_std.Asset_ID.map(dict_weights).astype('float32')
df_std.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,target,weights
0,1514764860,0,5,8.5300,8.5300,8.5300,8.5300,78.379997,8.530000,-0.014399,-0.015240,4.304065
1,1514764920,0,7,8.5300,8.5300,8.5145,8.5145,71.389999,8.520215,-0.015875,-0.014399,4.304065
2,1514764980,0,45,8.5065,8.5299,8.4848,8.4848,1546.819946,8.501393,-0.015410,-0.015875,4.304065
3,1514765040,0,14,8.5009,8.5066,8.4744,8.5009,125.800003,8.479810,-0.012524,-0.015410,4.304065
4,1514765100,0,5,8.5007,8.5007,8.4560,8.4560,125.010002,8.458435,-0.005940,-0.012524,4.304065


In [5]:
import time
from datetime import datetime
def timestamp_to_date(timestamp):
    return(datetime.fromtimestamp(timestamp))

ts = df_std.timestamp
ts = ts.apply(timestamp_to_date)

#%%
df_std['sin_month'] = (np.sin(2 * np.pi * ts.dt.month/12)).astype('float32')
df_std['cos_month'] = (np.cos(2 * np.pi * ts.dt.month/12)).astype('float32')
df_std['sin_day'] = (np.sin(2 * np.pi * ts.dt.day/31)).astype('float32')
df_std['cos_day'] = (np.cos(2 * np.pi * ts.dt.day/31)).astype('float32')
df_std['sin_hour'] = (np.sin(2 * np.pi * ts.dt.hour/24)).astype('float32')
df_std['cos_hour'] = (np.cos(2 * np.pi * ts.dt.hour/24)).astype('float32')
df_std['sin_minute'] = (np.sin(2 * np.pi * ts.dt.minute/60)).astype('float32')
df_std['cos_minute'] = (np.cos(2 * np.pi * ts.dt.minute/60)).astype('float32')
df_std

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,target,weights,sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute
0,1514764860,0,5,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399,-0.015240,4.304065,0.5,8.660254e-01,0.201299,0.979530,0.866025,-0.500000,0.104528,0.994522
1,1514764920,0,7,8.530000,8.530000,8.514500,8.514500,7.139000e+01,8.520215,-0.015875,-0.014399,4.304065,0.5,8.660254e-01,0.201299,0.979530,0.866025,-0.500000,0.207912,0.978148
2,1514764980,0,45,8.506500,8.529900,8.484800,8.484800,1.546820e+03,8.501393,-0.015410,-0.015875,4.304065,0.5,8.660254e-01,0.201299,0.979530,0.866025,-0.500000,0.309017,0.951057
3,1514765040,0,14,8.500900,8.506600,8.474400,8.500900,1.258000e+02,8.479810,-0.012524,-0.015410,4.304065,0.5,8.660254e-01,0.201299,0.979530,0.866025,-0.500000,0.406737,0.913545
4,1514765100,0,5,8.500700,8.500700,8.456000,8.456000,1.250100e+02,8.458435,-0.005940,-0.012524,4.304065,0.5,8.660254e-01,0.201299,0.979530,0.866025,-0.500000,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26080038,1632182160,13,477,0.091435,0.091500,0.091120,0.091221,2.613704e+06,0.091340,0.004163,0.004163,1.791759,-1.0,-1.836970e-16,-0.897805,-0.440394,0.965926,-0.258819,-0.406737,0.913545
26080039,1632182220,13,980,0.091131,0.091195,0.090802,0.090863,4.991572e+06,0.090991,0.004163,0.004163,1.791759,-1.0,-1.836970e-16,-0.897805,-0.440394,0.965926,-0.258819,-0.309017,0.951057
26080040,1632182280,13,720,0.090853,0.090940,0.090740,0.090859,3.699102e+06,0.090838,0.004163,0.004163,1.791759,-1.0,-1.836970e-16,-0.897805,-0.440394,0.965926,-0.258819,-0.207912,0.978148
26080041,1632182340,13,686,0.090941,0.091450,0.090860,0.091326,3.241507e+06,0.091193,0.004163,0.004163,1.791759,-1.0,-1.836970e-16,-0.897805,-0.440394,0.965926,-0.258819,-0.104528,0.994522


In [6]:
df=0
#del df_std['timestamp_std']
same_time_close=['same_0','same_1','same_2','same_3','same_4','same_5','same_6','same_7','same_8','same_9','same_10','same_11','same_12','same_13']
same_time_open=['same_0_open','same_1_open','same_2_open','same_3_open','same_4_open','same_5_open','same_6_open','same_7_open','same_8_open','same_9_open','same_10_open','same_11_open','same_12_open','same_13_open']
same_time_tar=['same_0_tar','same_1_tar','same_2_tar','same_3_tar','same_4_tar','same_5_tar','same_6_tar','same_7_tar','same_8_tar','same_9_tar','same_10_tar','same_11_tar','same_12_tar','same_13_tar']
same_time_high=['same_0_hig','same_1_hig','same_2_hig','same_3_hig','same_4_hig','same_5_hig','same_6_hig','same_7_hig','same_8_hig','same_9_hig','same_10_hig','same_11_hig','same_12_hig','same_13_hig']
same_time_low=['same_0_low','same_1_low','same_2_low','same_3_low','same_4_low','same_5_low','same_6_low','same_7_low','same_8_low','same_9_low','same_10_low','same_11_low','same_12_low','same_13_low']
same_time_count=['same_0_cou','same_1_cou','same_2_cou','same_3_cou','same_4_cou','same_5_cou','same_6_cou','same_7_cou','same_8_cou','same_9_cou','same_10_cou','same_11_cou','same_12_cou','same_13_cou']
same_time_vol=['same_0_vol','same_1_vol','same_2_vol','same_3_vol','same_4_vol','same_5_vol','same_6_vol','same_7_vol','same_8_vol','same_9_vol','same_10_vol','same_11_vol','same_12_vol','same_13_vol']
same_time_vwap=['same_0_vwa','same_1_vwa','same_2_vwa','same_3_vwa','same_4_vwa','same_5_vwa','same_6_vwa','same_7_vwa','same_8_vwa','same_9_vwa','same_10_vwa','same_11_vwa','same_12_vwa','same_13_vwa']
names=[same_time_close,same_time_open,same_time_tar,same_time_high,same_time_low,same_time_count,same_time_vol,same_time_vwap]
ecoph=0
for i in ["Close","Open","target","High","Low",'Count','Volume','VWAP']:#
    same_time_data=df_std[['timestamp','Asset_ID',i]].groupby(['timestamp','Asset_ID'])[i].sum().unstack()
    same_time_data.columns=names[ecoph]
    same_time_data=same_time_data.reset_index()
    ecoph+=1
    df_std=pd.merge(df_std,same_time_data,left_on='timestamp',right_on='timestamp',how='left')
    same_time_data=0
df_std.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,...,same_4_vwa,same_5_vwa,same_6_vwa,same_7_vwa,same_8_vwa,same_9_vwa,same_10_vwa,same_11_vwa,same_12_vwa,same_13_vwa
0,1514764860,0,5,8.5300,8.5300,8.5300,8.5300,78.379997,8.530000,-0.014399,...,NaN,7.657713,738.839294,25.891363,NaN,225.197937,NaN,329.454132,NaN,NaN
1,1514764920,0,7,8.5300,8.5300,8.5145,8.5145,71.389999,8.520215,-0.015875,...,NaN,7.656749,738.268982,25.896999,NaN,224.446625,NaN,329.450012,NaN,NaN
2,1514764980,0,45,8.5065,8.5299,8.4848,8.4848,1546.819946,8.501393,-0.015410,...,NaN,7.654267,737.994446,26.469513,NaN,225.980438,NaN,329.198975,NaN,NaN
3,1514765040,0,14,8.5009,8.5066,8.4744,8.5009,125.800003,8.479810,-0.012524,...,NaN,7.649660,737.303650,26.456736,NaN,224.552338,NaN,328.160004,NaN,NaN
4,1514765100,0,5,8.5007,8.5007,8.4560,8.4560,125.010002,8.458435,-0.005940,...,NaN,7.628076,736.163025,26.442987,NaN,224.032959,NaN,328.077789,NaN,NaN


In [7]:
df_std=df_std.fillna(method='bfill')
df_std.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,...,same_4_vwa,same_5_vwa,same_6_vwa,same_7_vwa,same_8_vwa,same_9_vwa,same_10_vwa,same_11_vwa,same_12_vwa,same_13_vwa
0,1514764860,0,5,8.5300,8.5300,8.5300,8.5300,78.379997,8.530000,-0.014399,...,0.002775,7.657713,738.839294,25.891363,2.263346,225.197937,1182.0,329.454132,0.405237,0.036604
1,1514764920,0,7,8.5300,8.5300,8.5145,8.5145,71.389999,8.520215,-0.015875,...,0.002775,7.656749,738.268982,25.896999,2.263346,224.446625,1182.0,329.450012,0.405237,0.036604
2,1514764980,0,45,8.5065,8.5299,8.4848,8.4848,1546.819946,8.501393,-0.015410,...,0.002775,7.654267,737.994446,26.469513,2.263346,225.980438,1182.0,329.198975,0.405237,0.036604
3,1514765040,0,14,8.5009,8.5066,8.4744,8.5009,125.800003,8.479810,-0.012524,...,0.002775,7.649660,737.303650,26.456736,2.263346,224.552338,1182.0,328.160004,0.405237,0.036604
4,1514765100,0,5,8.5007,8.5007,8.4560,8.4560,125.010002,8.458435,-0.005940,...,0.002775,7.628076,736.163025,26.442987,2.263346,224.032959,1182.0,328.077789,0.405237,0.036604


In [8]:
df=df_std[df_std['Asset_ID']==0].reset_index(drop=True)
df=df.loc[1800000:,]
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,...,same_4_vwa,same_5_vwa,same_6_vwa,same_7_vwa,same_8_vwa,same_9_vwa,same_10_vwa,same_11_vwa,same_12_vwa,same_13_vwa
1800000,1622764860,0,1180,428.220001,428.500000,427.000000,427.851990,4610.307129,427.769745,0.001338,...,0.399304,6.518707,2850.720459,69.582291,1.360867,192.870575,3652.094727,309.461792,0.424088,0.082034
1800001,1622764920,0,1063,428.054504,428.500000,426.609985,427.778015,3220.489502,427.582947,0.000761,...,0.398503,6.515410,2854.155273,69.653854,1.364171,193.146271,3658.551270,309.770050,0.423974,0.082093
1800002,1622764980,0,772,427.545502,428.010010,426.540009,426.780487,1995.195190,426.973846,0.001366,...,0.397409,6.510413,2847.726807,69.600136,1.360728,192.982590,3652.900391,308.728333,0.423354,0.082027
1800003,1622765040,0,1580,426.658997,426.769989,425.260010,425.886505,3307.573975,425.828033,0.002723,...,0.396541,6.489806,2840.660889,69.427063,1.356397,192.443802,3641.383057,308.347656,0.422110,0.081831
1800004,1622765100,0,1404,426.148987,426.760010,424.119995,424.618500,3810.603760,425.240173,0.000810,...,0.396275,6.471422,2836.947510,69.391052,1.354052,192.106613,3638.931885,310.900330,0.422292,0.081727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956955,1632182160,0,277,364.111511,364.200012,362.700012,362.917999,773.176025,363.458771,0.008493,...,0.207764,4.198326,2968.339355,49.120186,1.282960,157.300491,2428.051758,232.001144,0.282147,0.091340
1956956,1632182220,0,296,362.926514,363.200012,362.100006,362.200012,596.347778,362.655640,0.008493,...,0.207244,4.185219,2963.499268,48.927734,1.277524,156.741211,2420.973633,231.437561,0.281339,0.090991
1956957,1632182280,0,319,362.230988,362.399994,361.700012,361.858002,820.909729,361.962982,0.008493,...,0.206832,4.176891,2958.464844,48.838566,1.275548,156.356247,2419.701660,231.261963,0.280627,0.090838
1956958,1632182340,0,588,362.011993,364.299988,361.799988,364.114502,1268.055176,363.465607,0.008493,...,0.207613,4.191222,2970.867676,48.988327,1.279324,156.817444,2432.949951,231.955917,0.281377,0.091193


In [9]:
df['log_ret'] = np.log(df['Close']/df['Open'])
df['GK_vol'] = (1 / 2 * np.log(df['High'] / df['Low']) ** 2 - \
    (2 * np.log(2) - 1) * np.log(df['Close'] / df['Open']) ** 2).astype('float32')
df['RS_vol'] = np.log(df['High']/df['Close'])*np.log(df['High']/df['Open']) + \
    np.log(df['Low']/df['Close'])*np.log(df['Low']/df['Open'])

In [10]:
import math
labels=['Count','Open','High','Low','Close','Volume','VWAP']
for i in labels:
    for j in range(labels.index(i)+1,len(labels)):
        k=labels[j]
        df[i+'-'+k]=df[i]-df[k]
        df[i+'+'+k]=df[i]+df[k]
        df[i+'*'+k]=df[i]*df[k]
        df[i+'/'+k]=df[i]/df[k]
    df['log'+'_'+i]=np.log(df[i])
df['Gmv_avg']=df['Volume*VWAP']/df['Count']
#df['diff']=df['Open-Close']/df['High-Low']
others=same_time_close+same_time_open+same_time_high+same_time_low+same_time_count+same_time_vol+same_time_vwap
for k in others:
    df[k+'_log']=np.log(df[k])

#Asset_ID = pd.get_dummies(df['Asset_ID'], prefix='Asset_ID', prefix_sep='_')
#df=pd.concat([df,Asset_ID], axis=1)
df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,...,same_4_vwa_log,same_5_vwa_log,same_6_vwa_log,same_7_vwa_log,same_8_vwa_log,same_9_vwa_log,same_10_vwa_log,same_11_vwa_log,same_12_vwa_log,same_13_vwa_log
1800000,1622764860,0,1180,428.220001,428.500000,427.000000,427.851990,4610.307129,427.769745,0.001338,...,-0.918032,1.874676,7.955327,4.242510,0.308122,5.262019,8.203056,5.734835,-0.857814,-2.500619
1800001,1622764920,0,1063,428.054504,428.500000,426.609985,427.778015,3220.489502,427.582947,0.000761,...,-0.920039,1.874170,7.956531,4.243538,0.310547,5.263448,8.204823,5.735830,-0.858084,-2.499902
1800002,1622764980,0,772,427.545502,428.010010,426.540009,426.780487,1995.195190,426.973846,0.001366,...,-0.922790,1.873403,7.954277,4.242766,0.308020,5.262600,8.203277,5.732462,-0.859546,-2.500706
1800003,1622765040,0,1580,426.658997,426.769989,425.260010,425.886505,3307.573975,425.828033,0.002723,...,-0.924976,1.870233,7.951792,4.240277,0.304832,5.259804,8.200119,5.731228,-0.862489,-2.503100
1800004,1622765100,0,1404,426.148987,426.760010,424.119995,424.618500,3810.603760,425.240173,0.000810,...,-0.925646,1.867396,7.950484,4.239758,0.303102,5.258050,8.199446,5.739472,-0.862058,-2.504375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956955,1632182160,0,277,364.111511,364.200012,362.700012,362.917999,773.176025,363.458771,0.008493,...,-1.571353,1.434686,7.995758,3.894270,0.249170,5.058158,7.794845,5.446743,-1.265327,-2.393163
1956956,1632182220,0,296,362.926514,363.200012,362.100006,362.200012,596.347778,362.655640,0.008493,...,-1.573859,1.431559,7.994126,3.890344,0.244924,5.054596,7.791925,5.444310,-1.268193,-2.396999
1956957,1632182280,0,319,362.230988,362.399994,361.700012,361.858002,820.909729,361.962982,0.008493,...,-1.575849,1.429567,7.992426,3.888520,0.243376,5.052137,7.791399,5.443551,-1.270727,-2.398682
1956958,1632182340,0,588,362.011993,364.299988,361.799988,364.114502,1268.055176,363.465607,0.008493,...,-1.572082,1.432992,7.996609,3.891582,0.246332,5.055082,7.796860,5.446548,-1.268058,-2.394774


In [12]:
df=df.reset_index(drop=True)
df=df.replace([np.inf,-np.inf],np.nan)
df=df.fillna(method='pad')
df.to_csv(r"D:\Desktop\train_deal.csv",index=False)

In [21]:
df=df.reset_index(drop=True)
df=df.replace([np.inf,-np.inf],np.nan)
df=df.fillna(method='pad')

x_col=[x for x in df.columns if x not in ['Target','Asset_ID','timestamp','timestamp_std']]
X=df[x_col]
y=df['Target']
from sklearn.preprocessing import QuantileTransformer
from sklearn import preprocessing
quantile_transformer = preprocessing.QuantileTransformer(output_distribution='normal', random_state=10)
X = quantile_transformer.fit_transform(X)
#Xtest=quantile_transformer.fit_transform(Xtest)
#quantile_transformer.quantiles_ 
y=y.values

from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.3,random_state=2)
Xtrain

array([[ 1.90493705, -1.13303045, -1.13689036, ..., -0.641868  ,
        -0.37505431, -0.35840529],
       [-0.81265408,  1.13136474,  1.13143554, ...,  1.07053199,
         0.70057758,  0.79800577],
       [-2.11966823, -0.80682762, -0.81651079, ..., -1.12063576,
        -1.55918646, -1.33326115],
       ...,
       [ 0.75967739,  0.07508091,  0.07668751, ...,  0.50899916,
        -0.05482726,  0.12714989],
       [ 0.78674453,  0.3278946 ,  0.32822342, ...,  0.13117878,
         0.47173903,  0.45699958],
       [-0.96542031, -0.18656212, -0.19178922, ..., -0.19199671,
        -0.2288673 , -0.23106769]])

In [22]:
Xtrain.shape

(109872, 322)

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
import numpy as np
from tensorflow.keras.layers import Dropout
import warnings 
import tensorflow as tf
#warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM
adam = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.)

def baseline_model():
    # create model
    model = Sequential()
    #model.add(Dropout(0.2, input_shape=(72,)))
    model.add(LSTM(units=400, input_shape=(322,1), return_sequences=True,activation='relu'))
    model.add(LSTM(500, kernel_initializer='normal', return_sequences=True,activation='relu'))
#     model.add(LSTM(300, kernel_initializer='normal', return_sequences=True,activation='relu'))
    model.add(LSTM(400, kernel_initializer='normal', return_sequences=True,activation='relu'))
    model.add(LSTM(322, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=adam,metrics=['mae'])
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=500, verbose=2, validation_split=0.2)
lstm_Xtrain=np.reshape(Xtrain,(109872,322,1))
lstm_ytrain=np.reshape(ytrain,(109872,1,1))
estimator.fit(lstm_Xtrain,lstm_ytrain)
prediction=estimator.predict(np.array(Xtest))
pre=prediction.tolist()
plt.figure(figsize=(12,6))
plt.plot(np.array(ytest)[:100],label='history')
plt.plot(pre[:100],label='predict')
plt.legend()

Epoch 1/100
176/176 - 23887s - loss: 7.7992e-06 - mae: 0.0017 - val_loss: 7.9262e-06 - val_mae: 0.0017
Epoch 2/100
176/176 - 16026s - loss: 7.6307e-06 - mae: 0.0017 - val_loss: 7.9088e-06 - val_mae: 0.0017
Epoch 3/100
176/176 - 17400s - loss: 7.6434e-06 - mae: 0.0017 - val_loss: 7.8059e-06 - val_mae: 0.0017
Epoch 4/100
